In [2]:
import sqlite3
import numpy as np

In [3]:
# open the connection
db = sqlite3.connect('data/mydb')
cur = db.cursor()

In [21]:
# import feature matrix into a numpy matrix
# for now, pretend that we're only using data in the members table to build our features
cur.execute('''SELECT is_churn FROM training''')
y_train =  np.array([item[0] for item in cur.fetchall()])

In [23]:
y_train[:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [29]:
# select all the rows in the members dataset where the id is in the training dataset
cur.execute('''SELECT * FROM training 
            INNER JOIN members ON training.msno=members.msno''')
x_train = cur.fetchall()

In [32]:
x_train = np.array(x_train)

In [34]:
# there's a lot of rows missing (271468) but work with this for now
len(x_train)

699492

In [36]:
x_train.shape

(699492, 8)

In [13]:
[item[0] for item in y_train]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [6]:
y_train = np.array(y_train)

In [7]:
y_train

array([['ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=', '1'],
       ['f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=', '1'],
       ['zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=', '1'],
       ..., 
       ['eUa3xo16vpAjr43Cjlb6Kjf1NTILYyJIkBayJQdXWnw=', '0'],
       ['iZE41tbAQ65rJq60olkJT4BJzuUAYgQdfbEemXe/TTk=', '0'],
       ['oECkzJik4wKsbOEVY6UACLbmgM8qymFdb5cJaHrodY8=', '0']], 
      dtype='<U44')